In [3]:
%pip install pandas
%pip install matplotlib
%pip install reportlab

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
print(pd.__version__)

import matplotlib as mpl
print(mpl.__version__)

import reportlab
print(reportlab.__version__)

2.2.3
3.9.3
4.2.5


In [7]:
import pandas as pd 
import matplotlib.pyplot as plt 

# Importation for generate pdf 
from reportlab.lib.pagesizes import letter 
from reportlab.pdfgen import canvas 
from datetime import datetime 

# Variable that captures URL id 
sheets_id = "158i3c_YcNS2x8qxNv0YselYRqNVvG1gXjaFgkddHgvg" 
xsl = pd.ExcelFile(f"https://docs.google.com/spreadsheets/d/{sheets_id}/export?format=xlsx") 

# Read file 
spreadSheetsName = pd.read_excel(xsl, 'Semana_one', header=0) 

# Find the index where NaN values are present to separate data 
separator_index = spreadSheetsName[spreadSheetsName.isna().all(axis=1)].index[0] 

# Separate data dynamically based on the NaN row 
vendas = spreadSheetsName.iloc[:separator_index, :] 
cancelamentos = spreadSheetsName.iloc[separator_index+1:, :] 

# Rename columns for cancelamentos DataFrame 
cancelamentos.columns = ['Dias', 'Cancelamentos'] 

# Convert 'Cancelamentos' column to numeric, forcing errors to NaN and then filling with 0 
cancelamentos.loc[:, 'Cancelamentos'] = pd.to_numeric(cancelamentos['Cancelamentos'], errors='coerce').fillna(0).astype(int) 

def create_graph(data, coloring, xLabelText, yLabelText, title, filename): 

    plt.figure(figsize=(10, 5)) 
    plt.bar(data[xLabelText], data[yLabelText], color=coloring) 
    plt.xlabel(xLabelText) 
    plt.ylabel(yLabelText) 
    plt.title(title) 
    plt.xticks(rotation=45, ha='right')  
    plt.tight_layout() 
    plt.savefig(filename) 
    plt.close() 

def create_pdf():

    # Create a PDF document
    pdf_filename = 'weecly-sales.pdf' 
    c = canvas.Canvas(pdf_filename, pagesize=letter)

    # Create and save sales chart
    create_graph(vendas, 'blue', 'Dias', 'Vendas', 'Vendas por dia', 'sales_chart.png') 

    # Create and save cancellations chart
    create_graph(cancelamentos, 'red', 'Dias', 'Cancelamentos', 'Cancelamentos por dia', 'cancellations_chart.png') 

    # Draw sales chart on PDF
    c.drawImage('sales_chart.png', 10, 400, width=500, height=300) 

    # Draw cancellations chart on PDF
    c.drawImage('cancellations_chart.png', 10, 100, width=500, height=300) 

    # Add text or elements to the PDF
    c.drawString(10, 700, "Sales Report - Semana One") 
    c.drawString(10, 680, "Generated on: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S")) 

    # Save the PDF
    c.save() 

    print(f"PDF generated successfully: {pdf_filename}") 

create_pdf()

PDF generated successfully: weecly-sales.pdf
